In [1]:
import pandas as pd
from functions import assembleDf, epochElo, epochsElo, epochG, epochsG,PlayerElo
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [3]:
singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_11425/3293468424.py:1: DtypeWarning: Columns (3,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'],


In [4]:
playerClasses, ratingsHistory = epochsElo(matches)

In [6]:
ratingsHistory_df = assembleDf(ratingsHistory)

In [7]:
# fill in missing values with last seen rating.  
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)

In [14]:
def get_recent_rating_wp(ratingsHistory_df,tourney_date,winner_id,loser_id):
    if not type(tourney_date) == pd._libs.tslibs.timestamps.Timestamp:
        tourney_date = pd.to_datetime(tourney_date)
    timestamp = max(ratingsHistory_df.index[ratingsHistory_df.index<=tourney_date])
    winner_rating, loser_rating = ratingsHistory_df.loc[timestamp,[winner_id,loser_id]]
    wp = 1/(1 + 10**((loser_rating-winner_rating)/400))
    return winner_rating, loser_rating, wp

In [15]:
get_recent_rating_wp(ratingsHistory_df,pd.Timestamp('1877-07-09T00'),131867,131879)

(1500.0, 1500.0, 0.5)

In [9]:
padRow = pd.DataFrame({col: 1500.0 for col in ratingsHistory_df.columns}, index = [pd.Timestamp('1877-07-09T00')])
padRow

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0


In [10]:
ratingsHistory_df = pd.concat([padRow,ratingsHistory_df],axis=0)

In [20]:
we_le_pw = matches.apply(lambda x:
                           get_recent_rating_wp(
                               ratingsHistory_df, 
                               x['tourney_date'], 
                               x['winner_id'], 
                               x['loser_id']),
                            axis=1)

In [ ]:
singles_r_wp = pd.concat(we_le_pw_df,pd.DataFrame(matches.apply(lambda x: 
                           get_recent_rating_wp(
                               ratingsHistory_df, 
                               x['tourney_date'], 
                               x['winner_id'], 
                               x['loser_id']),
                            axis=1),columns = ['winner_elo','loser_elo','predicted_wins'])])

In [ ]:
singles_matches[1:10].shape,matches[1:10].shape

In [ ]:
matches.dtypes

In [ ]:
testDf

In [ ]:
matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)

In [ ]:
testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)

In [ ]:
testDf = singles_matches[1:10]
testDf['winner_elo','loser_elo','predicted_wins'] = matches[1:10].apply(
    lambda x: get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], 
                                   x['winner_id'], x['loser_id']),axis=1)


In [ ]:
[1,2,3][0:-1]

In [ ]:
ratingsHistory_df

In [ ]:
singles_matches

In [ ]:
ratingsHistory_df.columns

In [ ]:
set(list(matches['winner_id'].unique()) + list(matches['loser_id'].unique()))

In [ ]:
set(ratingsHistory_df) - 

In [ ]:
singles_matches[['winner_elo','loser_elo','predicted_wins']] = matches[['tourney_date','winner_id','loser_id']].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=1)

In [ ]:
ratingsHistory_df